In [16]:
from smartapi import SmartConnect, SmartWebSocket
import time
from datetime import datetime
import datetime
import pyotp

In [17]:
apiKey='gmPdFX9J'
userName='A1221496'
password='0513'  #This is pin not the password of AngelOne
totp='CPEXLLRYLGP3YT4553O4OCXB64'

In [18]:
obj=SmartConnect(api_key=apiKey)
token=pyotp.TOTP(totp).now()

data = obj.generateSession(userName,password,token)


refreshToken= data['data']['refreshToken']

#print("refresh token: "+refreshToken)

feedToken=obj.getfeedToken()
#print("feed token: "+feedToken)

#fetch User Profile
userProfile= obj.getProfile(refreshToken)
#print(userProfile)

In [19]:
obj.position()

{'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': None}

In [27]:
import pandas as pd
import requests
import numpy as np
import math

url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
d = requests.get(url).json()
token_df = pd.DataFrame.from_dict(d)
token_df['expiry'] = pd.to_datetime(token_df['expiry'])
token_df = token_df.astype({'strike': float})

In [28]:
list(token_df)

['token',
 'symbol',
 'name',
 'expiry',
 'strike',
 'lotsize',
 'instrumenttype',
 'exch_seg',
 'tick_size']

In [29]:
token_df[token_df["symbol"] == "BANKNIFTY"]

,token,symbol,name,expiry,strike,lotsize,instrumenttype,exch_seg,tick_size
4659,26009,BANKNIFTY,BANKNIFTY,NaT,-1.0,-1,,NSE,-1.000000


In [30]:
 LTP=obj.ltpData("NSE","BANKNIFTY","26009")
 

In [33]:
ltp = LTP['data']['ltp']
ltp

41350.4

In [53]:
ATMstrike1 = math.floor(ltp/100)*100
ATMstrike1



41300

In [54]:
ATMstrike2 = math.ceil(ltp/100)*100
ATMstrike2

41400

In [55]:
if ltp-ATMstrike1 <= ATMstrike2-ltp:
    ATMstrike = ATMstrike1
else:
    ATMstrike = ATMstrike2
    

In [56]:
ATMstrike

41300

In [64]:
def place_order(token,symbol,quantity,buy_sell,price):
    try:
        orderparams = {
        "variety": "NORMAL",
        "tradingsymbol": symbol,
        "symboltoken": token,
        "transactiontype": buy_sell,
        "exchange": "NFO",
        "ordertype": "MARKET",
        "producttype": "INTRADAY",
        "duration": "DAY",
        "price": price,
        "squareoff": "0",
        "stoploss": "0",
        "quantity": quantity
        }
        orderId=obj.placeOrder(orderparams)
        print("The order id is: {}".format(orderId))
    except Exception as e:
        print("Order placement failed: {}".format(e.message))


In [65]:
def getTokenInfo (df, exch_seg, instrumenttype,symbol,strike_price,pe_ce):
    #df = credentials.TOKEN_MAP
    strike_price = strike_price*100
    if exch_seg == 'NSE':
        eq_df = df[(df['exch_seg'] == 'NSE') & (df['symbol'].str.contains('EQ')) ]
        return eq_df[eq_df['name'] == symbol]
    elif exch_seg == 'NFO' and ((instrumenttype == 'FUTSTK') or (instrumenttype == 'FUTIDX')):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol)].sort_values(by=['expiry'])
    elif exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce))].sort_values(by=['expiry'])

In [66]:
ce_strike = getTokenInfo(token_df,'NFO','OPTIDX','BANKNIFTY',ATMstrike,'CE').iloc[0]
ce_strike

token                               45974
symbol            BANKNIFTY09MAR2341300CE
name                            BANKNIFTY
expiry                2023-03-09 00:00:00
strike                          4130000.0
lotsize                                25
instrumenttype                     OPTIDX
exch_seg                              NFO
tick_size                        5.000000
Name: 60770, dtype: object

In [67]:
pe_strike = getTokenInfo(token_df,'NFO','OPTIDX','BANKNIFTY',ATMstrike,'PE').iloc[0]
pe_strike

token                               45975
symbol            BANKNIFTY09MAR2341300PE
name                            BANKNIFTY
expiry                2023-03-09 00:00:00
strike                          4130000.0
lotsize                                25
instrumenttype                     OPTIDX
exch_seg                              NFO
tick_size                        5.000000
Name: 41799, dtype: object

In [68]:
place_order(ce_strike['token'],ce_strike['symbol'],ce_strike['lotsize'],'SELL',0)

The order id is: 230306000940083


In [69]:
place_order(pe_strike['token'],pe_strike['symbol'],pe_strike['lotsize'],'SELL',0)

The order id is: 230306000940086
